<a href="https://colab.research.google.com/github/andrePankraz/qa_service/blob/main/notebooks/Contriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test Contriever

From here: https://github.com/facebookresearch/contriever

Model is used for document retrieval. Compareable sentence embeddings are created.

In [ ]:
!pip install --quiet sentence_cleaner_splitter@git+https://github.com/facebookresearch/LASER.git#subdirectory=utils 

  Preparing metadata (setup.py) ... done


Contriever isn't a Python package. Must clone project and import module via sys-path.

In [ ]:
!git clone https://github.com/facebookresearch/contriever.git

fatal: destination path 'contriever' already exists and is not an empty directory.


In [ ]:
import os
import sys
sys.path.append(os.path.abspath('/content/contriever'))

from src.contriever import Contriever
from transformers import AutoTokenizer

contriever = Contriever.from_pretrained("facebook/mcontriever-msmarco") 
tokenizer = AutoTokenizer.from_pretrained("facebook/mcontriever-msmarco")

Create sentence embeddings:

In [ ]:
sentences = [
    "Where was Marie Curie born?",
    "Maria Sklodowska, later known as Marie Curie, was born on November 7, 1867.",
    "Born in Paris on 15 May 1859, Pierre Curie was the son of Eugène Curie, a doctor of French Catholic origin from Alsace."
]

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
embeddings = contriever(**inputs)

In [ ]:
embeddings.shape


torch.Size([3, 768])

In [ ]:
embeddings[0] @ embeddings[1] #1.0473

tensor(3.2272, grad_fn=<DotBackward0>)

In [ ]:
embeddings[0] @ embeddings[2] #1.0095

tensor(3.1328, grad_fn=<DotBackward0>)

In [ ]:
from sentence_transformers import util

query_embedding = contriever(**tokenizer(['Wie heißt die Hauptstadt von Deutschland?'], padding=True, truncation=True, return_tensors="pt"))
passage_embedding = contriever(**tokenizer(['Die Hauptstadt von Deutschland ist Berlin.',
                                    'In London, der Hauptstadt von GB, wohnen auch viele Deutsche.',
                                    'Da beißt die Maus keinen Faden ab.'], padding=True, truncation=True, return_tensors="pt"))

print(f"Type: {type(query_embedding)}, Shape: {query_embedding.shape}")
print("Dot-Similarity:", util.dot_score(query_embedding, passage_embedding))
print("Cos-Similarity:", util.cos_sim(query_embedding, passage_embedding))

Type: <class 'torch.Tensor'>, Shape: torch.Size([1, 768])
Dot-Similarity: tensor([[2.5570, 1.4620, 0.9565]], grad_fn=<MmBackward0>)
Cos-Similarity: tensor([[0.8060, 0.4456, 0.2775]], grad_fn=<MmBackward0>)


In [ ]:
question = 'Wer ist der Bürgermeister von Dresden?'
title = 'Dresden'
website_url = 'https://de.wikipedia.org/wiki/' + title

In [ ]:
import requests

res = requests.get(website_url)
page_html = res.content

# html_page

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page_html, features="html.parser")
page_lines = []
line = ''
for e in soup.descendants:
    if isinstance(e, str):
        line += e.replace(u'\xa0', u' ')
    elif e.name in ['br', 'p', 'h1', 'h2', 'h3', 'h4', 'tr', 'th']:
        if len(line) > 50:
          page_lines.append(line)
        line = ''
    elif e.name == 'li':
        if len(line) > 50:
          page_lines.append('- ' + line)
        line = ''
page_text = '\n'.join(page_lines)
print(page_text)

html



Dresden – Wikipedia
document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t.",".\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","Januar","Februar","März","April","Mai","Juni","Juli","August","September","Oktober","November","Dezember"],"wgRequestId":"e3af2919-0d31-4916-94e2-db917e04e245","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Dresden","wgTitle":"Dresden","wgCurRevisionId":229911472,"wgRevisionId":229911472,"wgArticleId":1084,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia:Defekte Weblinks/Ungeprüfte Archivlinks 2022-10","Wikipedia:Überarbeiten","Wikipedia:Veraltet nach Jahr 2023","Dresden","Gemeinde in Sachsen","Wikipedia:Gesprochener Artikel","Wikipedia:Exzellent","Kreisfreie Stadt in Sachsen","Deutsche Landeshauptstadt","Ort mit Binnen

In [ ]:
from sentence_cleaner_splitter.cleaner_splitter import SentenceSplitClean

sentence_splitter = SentenceSplitClean('de', 'default')
split_text = ''
sentences = []
for _, _, line in sentence_splitter(page_text):
  if len(line) > 50:
    sentences.append(f"{title}: {line.strip()}" if title not in line else line.strip())
embeddings = contriever(**tokenizer(sentences, padding=True, truncation=True, return_tensors="pt"))

print(len(embeddings))

| 2023-01-20 09:31:08,743 | INFO | khmer-nltk | Loaded model from /usr/local/lib/python3.8/dist-packages/khmernltk/word_tokenize/sklearn_crf_ner_10000.sav |
INFO:khmer-nltk:Loaded model from /usr/local/lib/python3.8/dist-packages/khmernltk/word_tokenize/sklearn_crf_ner_10000.sav
| 2023-01-20 09:31:08,785 | INFO | khmer-nltk | Loaded model from /usr/local/lib/python3.8/dist-packages/khmernltk/pos_tag/sklearn_crf_pos_alt_0.9846.sav |
INFO:khmer-nltk:Loaded model from /usr/local/lib/python3.8/dist-packages/khmernltk/pos_tag/sklearn_crf_pos_alt_0.9846.sav
